In [1]:
%cd ../

/u02/thanhhm/research-idea


In [2]:
import plasma.parallel_processing as pp
import time

In [3]:
def compute(data):
    time.sleep(1)
    return f'{data:05d}'

def print_pass(data):
    print(data)
    return data

In [4]:
class Caller(pp.communicators.Flow):

    def __init__(self):
        super().__init__()
        manager = pp.mp.Manager()

        self.computation = compute
        self.printer = print_pass
        self.aggregator = pp.communicators.Aggregator(20, 0.5, manager)
        self.registerIOs(computation=pp.queues.ProcessQueue(4), printer=pp.queues.ProcessQueue(), aggregator=pp.queues.ProcessQueue())

caller = Caller()
caller

computation-<function compute at 0x7f342c6f7ce0>
printer-<function print_pass at 0x7f342c6f7d80>
aggregator-<class 'plasma.parallel_processing.communicators.aggregator.Aggregator'>

In [5]:
caller.run()

In [6]:
for i in range(20):
    caller.put(i)

In [7]:
caller.aggregator.wait()

  0%|          | 0/20 [00:00<?, ?it/s]

00000
00003
00001
00002
00004
00005
00006
00007
00009
00008
00010
00011
00012
00013
00014
00015
00016
00017
00018
00019


In [8]:
caller.aggregator.results

['00000',
 '00003',
 '00001',
 '00002',
 '00004',
 '00005',
 '00006',
 '00007',
 '00009',
 '00008',
 '00010',
 '00011',
 '00012',
 '00013',
 '00014',
 '00015',
 '00016',
 '00017',
 '00018',
 '00019']

In [9]:
caller.release()

In [10]:
caller.aggregator

Aggregator(
	finished=0,
	total=20,
	sleep=0.5,
	ignore_none=True,
)